In [1]:
import datetime
import requests
import hyp3_sdk
import numpy as np
from tqdm import tqdm 
from copy import deepcopy

In [2]:
%load_ext autoreload
%autoreload 2

In [355]:
def isce_jobs4resubmit(jobs_dicts:list):
    job_definition = {'name': None,
                      'job_parameters': None,
                      'job_type': 'INSAR_ISCE'
                     }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)
        prepared_job['name'] = job['name']
        prepared_job['job_parameters'] = job['job_parameters']
        prepared_job['job_type'] = job['job_type']
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def raider_jobs4resubmit(jobs_dicts:list):
    job_definition = {
        'name': 're-run RAiDER step',  # optional: provide a name
        "job_type": "ARIA_RAIDER",
        "job_parameters": {
            # "job_id": "27836b79-e5b2-4d8f-932f-659724ea02c3",
            # "weather_model": "ERA5"
        },
    }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)
        prepared_job['name'] = job['name'] + '_rerun_raider'
        if job['job_type'] == 'INSAR_ISCE':
            prepared_job['job_parameters']['job_id'] = job['job_id']
        else:
            prepared_job['job_parameters']['job_id'] = job['job_parameters']['job_id']
        prepared_job['job_parameters']['weather_model'] = job['job_parameters']['weather_model'] 
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def split_failed_jobs(failed_jobs):
    flag_insar_job = lambda x, y: (x.job_type == 'INSAR_ISCE') & (len(x.processing_times)==y)
    # isce
    isce_jobs = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 1)]
    # raider
    raider_jobs_new = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 2)] 
    raider_jobs = [j.to_dict() for j in failed_jobs if j.job_type == 'ARIA_RAIDER']
    raider_jobs = raider_jobs_new + raider_jobs
    return isce_jobs, raider_jobs

def get_request_date(job_dict):
    job_date = datetime.datetime.strptime(job_dict['request_time'], '%Y-%m-%dT%H:%M:%S%z')
    return datetime.datetime.strftime(job_date,'%Y-%m-%d')  

# TODO: add function to find duplicate jobs to filter out same jobs
# for now track it by date
class access_hyp3_resubmit():
    def __init__(self, hyp3_jobs):
        # Filter to failed jobs
        failed_jobs = hyp3_jobs.filter_jobs(succeeded=False, pending=False,
                                            running=False, failed=True)
        print(f'Failed jobs:\n {failed_jobs}')

        # Failed at topssApp step
        self.isce_jobs, self.raider_jobs = split_failed_jobs(failed_jobs)
        print(f'Topsapp jobs: {len(self.isce_jobs)}')
        print(f' Submission dates: {np.unique(list(map(get_request_date, self.isce_jobs)))}')

        # Failed at raider step
        print(f'Raider jobs: {len(self.raider_jobs)}')
        print(f' Submission dates: {np.unique(list(map(get_request_date, self.raider_jobs)))}')

    def prepare_jobs(self, batch_size=200):
        self.prepared_isce = isce_jobs4resubmit(self.isce_jobs)
        self.prepared_raider = raider_jobs4resubmit(self.raider_jobs)
        # isce
        if len(self.prepared_isce) > 0:
            self.isce_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_isce, batch_size)]
            msg = f'Prepared isce jobs: {len(self.prepared_isce)} in {len(self.isce_resubmit[0])}'
            msg += f'x {len(self.isce_resubmit)} batches'
            print(msg)
        else:
            self.isce_resubmit = None 
        # raider
        if len(self.prepared_raider) > 0:
            self.raider_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_raider, batch_size)]
            msg = f'Prepared raider jobs: {len(self.prepared_raider)} in {len(self.raider_resubmit[0])}'
            msg += f'x {len(self.raider_resubmit)} batches'
            print(msg)
        else:
            self.raider_resubmit = None
        
    def submit_isce(self):
        if self.isce_resubmit is None:
            raise ValueError('There are no prepared jobs!') 
        print(f'Submitting: {len(self.prepared_isce)} jobs')
        for batch in tqdm(self.isce_resubmit):
            hyp3_isce.submit_prepared_jobs(batch)

    def submit_raider(self):
        if self.raider_resubmit is None:
            raise ValueError('There are no prepared jobs!') 
        print(f'Submitting: {len(self.prepared_raider)} jobs')
        for batch in tqdm(self.raider_resubmit):
            hyp3_isce.submit_prepared_jobs(batch) 


## Setup

In [4]:
# uses .netrc; add `prompt=True` to prompt for credentials;
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu', prompt=True)
# hyp3_isce = hyp3_sdk.HyP3('https://hyp3-tibet-jpl.asf.alaska.edu', prompt=True)

In [244]:
filtering_flags = dict(succeeded=False, pending=False,
                        running=False, failed=True)

In [257]:
# Since the campaign first date
jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 3, 18), 
                           end=datetime.datetime(2024, 3, 21),
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

97689 HyP3 Jobs: 32489 succeeded, 65200 failed, 0 running, 0 pending.
##########
Failed jobs:
 65200 HyP3 Jobs: 0 succeeded, 65200 failed, 0 running, 0 pending.
Topsapp jobs: 19074
 Sumbission dates: ['2024-03-20']
Raider jobs: 45775
 Sumbission dates: ['2024-03-20']


In [261]:
# Get unique job names
np.unique([j['name'] for j in jobs.isce_jobs])

array(['conus_west_coast-track100', 'conus_west_coast-track108',
       'conus_west_coast-track115', 'conus_west_coast-track13',
       'conus_west_coast-track137', 'conus_west_coast-track144',
       'conus_west_coast-track144_1', 'conus_west_coast-track166',
       'conus_west_coast-track166_1', 'conus_west_coast-track173',
       'conus_west_coast-track35', 'conus_west_coast-track42',
       'conus_west_coast-track64', 'conus_west_coast-track64_1',
       'conus_west_coast-track71', 'conus_west_coast-track86',
       'conus_west_coast-track93'], dtype='<U27')

In [297]:
# Since I started
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 4, 29), 
                           end=datetime.datetime(2024, 5, 6),
                           user_id='access_cloud_based_insar',
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 4, 29), 
                           end=datetime.datetime(2024, 5, 6),
                           job_type='ARIA_RAIDER')
print('#' * 10)
print('Raider')
print(raider_jobs)

21512 HyP3 Jobs: 12758 succeeded, 8754 failed, 0 running, 0 pending.
##########
Failed jobs:
 8754 HyP3 Jobs: 0 succeeded, 8754 failed, 0 running, 0 pending.
Topsapp jobs: 4639
 Sumbission dates: ['2024-05-01' '2024-05-05']
Raider jobs: 4115
 Sumbission dates: ['2024-05-01' '2024-05-05']
##########
Raider
50374 HyP3 Jobs: 26043 succeeded, 24331 failed, 0 running, 0 pending.


In [308]:
# unique succeded aria_raider jobs
np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**{**filtering_flags, **{'succeeded':True, 'failed':False}}).jobs]).shape

(26043,)

In [310]:
# unique failed aria_raider jobs
np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**filtering_flags).jobs]).shape

(24013,)

In [267]:
# Get unique job names
np.unique([j['name'] for j in jobs.isce_jobs])

array(['conus_west_coast-track100', 'conus_west_coast-track108',
       'conus_west_coast-track115', 'conus_west_coast-track13',
       'conus_west_coast-track137', 'conus_west_coast-track144',
       'conus_west_coast-track144_1', 'conus_west_coast-track166',
       'conus_west_coast-track166_1', 'conus_west_coast-track173',
       'conus_west_coast-track35', 'conus_west_coast-track42',
       'conus_west_coast-track64', 'conus_west_coast-track64_1',
       'conus_west_coast-track71', 'conus_west_coast-track86',
       'conus_west_coast-track93'], dtype='<U27')

In [270]:
print(f'Succeded jobs: {32489+12758+26046}') # till 2024-04-29
print(f'Missing: {97689 - (32489+12758+26046)}')

# breakdown
# isce of 19074 failed: succeded: 17397 missing: 1677
# raider of 45775 + 4115 failed: succedded: 26043 missing: 23847

Succeded jobs: 71293
Missing: 26396


In [364]:
# Last run
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 6), 
                           end=datetime.datetime(2024, 5, 21),
                           user_id='access_cloud_based_insar',
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 5, 6), 
                           end=datetime.datetime(2024, 5, 21),
                           job_type='ARIA_RAIDER')
print('#' * 10)
print('Raider')
print(raider_jobs)

3329 HyP3 Jobs: 129 succeeded, 1133 failed, 939 running, 1128 pending.
##########
Failed jobs:
 1133 HyP3 Jobs: 0 succeeded, 1133 failed, 0 running, 0 pending.
Topsapp jobs: 1128
 Sumbission dates: ['2024-05-07']
Raider jobs: 3
 Sumbission dates: ['2024-05-07']
##########
Raider
73190 HyP3 Jobs: 3 succeeded, 48856 failed, 42 running, 24289 pending.


In [345]:
raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                                  start=datetime.datetime(2024, 5, 4), 
                                  end=datetime.datetime(2024, 5, 6),
                                  job_type='ARIA_RAIDER')

In [346]:
print(raider_jobs)

50373 HyP3 Jobs: 26042 succeeded, 24331 failed, 0 running, 0 pending.


In [356]:
r = access_hyp3_resubmit(raider_jobs.filter_jobs(**filtering_flags))

Failed jobs:
 24331 HyP3 Jobs: 0 succeeded, 24331 failed, 0 running, 0 pending.
Topsapp jobs: 0
 Sumbission dates: []
Raider jobs: 24331
 Sumbission dates: ['2024-05-05']


In [357]:
r.prepare_jobs()

Prepared raider jobs: 24331 in 200x 122 batches


In [359]:
r.submit_raider()

Submitting: 24331 jobs


100%|██████████| 122/122 [05:37<00:00,  2.77s/it]


In [317]:
# unique  aria_raider jobs
np.unique([j.job_parameters['job_id'] for j in raider_jobs.jobs]).shape

(24331,)

In [318]:
np.unique(list(map(get_request_date, [j.to_dict() for j in raider_jobs.jobs])))

array(['2024-05-07'], dtype='<U10')

## Re-submission

Track jobs by the submission dates

In [366]:
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 7), 
                           end=datetime.datetime(2024, 5, 21),
                           user_id='access_cloud_based_insar')
print(jobs)

76519 HyP3 Jobs: 269 succeeded, 50019 failed, 1033 running, 25198 pending.


In [323]:
# Example to split jobs that failed on topsaap and on raider step
fisce, fraider = split_failed_jobs(jobs.filter_jobs(**filtering_flags))
len(fisce), len(fraider)

(1039, 27160)

In [361]:
rjobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

Failed jobs:
 49989 HyP3 Jobs: 0 succeeded, 49989 failed, 0 running, 0 pending.
Topsapp jobs: 1128
 Sumbission dates: ['2024-05-07']
Raider jobs: 48859
 Sumbission dates: ['2024-05-07']


In [328]:
# to check for unique job_ids
np.unique([j['job_parameters']['job_id'] for j in rjobs.raider_jobs]).shape

(24328,)

In [362]:
rjobs.prepare_jobs()

Prepared isce jobs: 1128 in 200x 6 batches
Prepared raider jobs: 48859 in 200x 245 batches


In [363]:
rjobs.submit_isce()

Submitting: 1128 jobs


100%|██████████| 6/6 [00:12<00:00,  2.05s/it]


In [ ]:
rjobs.submit_raider()

In [337]:
succeded_flag = {**filtering_flags, **{'succeeded':True, 'failed':False}}
unique_succeded = np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**succeded_flag).jobs])
unique_failed = np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**filtering_flags).jobs])
# NOTE compare ARIA_RAIDER with INSAR_ISCE job_id

In [341]:
raider_jobs.filter_jobs(**succeded_flag).jobs

[Job.from_dict({'job_id': 'fe41e4ec-df22-4c09-8db8-c5e17bd547d8', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T16:01:43+00:00', 'status_code': 'SUCCEEDED', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track115_rerun_raider_rerun_raider', 'job_parameters': {'job_id': 'c84ea0ec-6f85-45d0-83b4-2918714d4f28', 'weather_model': 'HRRR'}, 'files': [{'filename': 'S1-GUNW-D-R-115-tops-20210807_20160908-141353-00123W_00043N-PP-6d76-v3_0_1.nc', 's3': {'bucket': 'hyp3-a19-jpl-contentbucket-1wfnatpznlg8b', 'key': 'fe41e4ec-df22-4c09-8db8-c5e17bd547d8/S1-GUNW-D-R-115-tops-20210807_20160908-141353-00123W_00043N-PP-6d76-v3_0_1.nc'}, 'size': 146907363, 'url': 'https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/fe41e4ec-df22-4c09-8db8-c5e17bd547d8/S1-GUNW-D-R-115-tops-20210807_20160908-141353-00123W_00043N-PP-6d76-v3_0_1.nc'}], 'logs': [], 'browse_images': [], 'thumbnail_images': [], 'expiration_time': '2024-11-04T00:00:00+00:00', 'processing_times':

In [342]:
#rerun_raider_rerun_raider_rerun_raider
raider_jobs.jobs

[Job.from_dict({'job_id': 'bb1f2a0d-bd2b-47eb-94e7-5413b947e2a5', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T19:48:02+00:00', 'status_code': 'PENDING', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track115_rerun_raider_rerun_raider_rerun_raider', 'job_parameters': {'job_id': 'fc2b52e7-9795-4d8a-b15e-f880930d2329', 'weather_model': 'HRRR'}, 'credit_cost': 1}),
 Job.from_dict({'job_id': 'd6697b57-1b82-455f-a42e-cb5053f23cdc', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T19:48:02+00:00', 'status_code': 'PENDING', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track166_1_rerun_raider_rerun_raider_rerun_raider', 'job_parameters': {'job_id': '7d21a75a-0093-490a-934c-358630733427', 'weather_model': 'HRRR'}, 'credit_cost': 1}),
 Job.from_dict({'job_id': '9e37d8e6-6b52-48ce-9f9a-43d6fd03f64c', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T19:48:02+00:00', 'status_code': 'PENDING', 'user_id': 'access_cloud_based_insar', 'name':

In [340]:
# Could filter wrong raider submission with processing time as they failed right away, or with name rerun_raider_rerun_raider_rerun_raider as each submisson adds rerun_raider 
raider_jobs.filter_jobs(**filtering_flags).jobs

[Job.from_dict({'job_id': '728d8e19-7362-4f02-bcf5-68f05d885d13', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T18:49:59+00:00', 'status_code': 'FAILED', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track35_rerun_raider_rerun_raider_rerun_raider', 'job_parameters': {'job_id': '24eeb3a9-3e2e-4e0c-9068-7af18b6fcf3b', 'weather_model': 'HRRR'}, 'logs': ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/728d8e19-7362-4f02-bcf5-68f05d885d13/728d8e19-7362-4f02-bcf5-68f05d885d13.log'], 'expiration_time': '2024-11-04T00:00:00+00:00', 'processing_times': [4.589], 'credit_cost': 1}),
 Job.from_dict({'job_id': '5e1388d2-ff62-4b6d-87f3-de511d1f8cb1', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-07T18:49:59+00:00', 'status_code': 'FAILED', 'user_id': 'access_cloud_based_insar', 'name': 'conus_west_coast-track35_rerun_raider_rerun_raider_rerun_raider', 'job_parameters': {'job_id': '6377a14b-db9e-4fb4-9b57-e301a709c899', 'weather_model': '

In [338]:
unique_failed.shape

(24328,)

In [336]:
len(raider_jobs.filter_jobs(**succeded_flag).jobs), unique_succeded.shape

(3, (3,))

In [330]:
## Check failed jobs
print(f'# failed jobs {len(rjobs.raider_jobs)}')
all_logs = [j['logs'] for j in rjobs.raider_jobs]
all_logs[0], all_logs[-1]

# failed jobs 27160


(['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/91d2ad06-b84c-4d88-bfdf-b5881c3684ed/91d2ad06-b84c-4d88-bfdf-b5881c3684ed.log'],
 ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/9d27256a-acf3-4ed9-92b0-c0d04374a6c8/9d27256a-acf3-4ed9-92b0-c0d04374a6c8.log'])

In [135]:
# Get log last 3 lines
'''
failed_logs = []
for ix, log in enumerate(all_logs):
    r = requests.get(log)
    failed_logs.append({ix: r.text.splitlines()[-3:]})'''

'\nfailed_logs = []\nfor ix, log in enumerate(all_logs):\n    r = requests.get(log)\n    failed_logs.append({ix: r.text.splitlines()[-3:]})'

## asf search

In [136]:
import asf_search as asf
import geopandas as gpd

In [201]:
# requires asf_search > 7.0.8
opts = asf.ASFSearchOptions(
    #shortName='ARIA_S1_GUNW', # version 3
    #relativeOrbit=[13],
    #flightDirection='DESCENDING',
    processingLevel='GUNW_STD',
    intersectsWith='POLYGON((-127.992 32.2865,-115.5115 32.2865,-115.5115 49.1812,-127.992 49.1812,-127.992 32.2865))',
    collectionAlias=False,
)

In [202]:
scenes = asf.search(opts=opts)
gdf = gpd.GeoDataFrame.from_features(scenes.geojson(), crs='EPSG:4326')

In [209]:
gdf.shape

(67249, 30)

In [210]:
version = gdf.fileID.apply(lambda x: int(x.split('-')[-1].split('_')[0].split('v')[-1]))
version.unique()

array([2])